In [224]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [243]:
class shortGridWorld():
    def __init__(self, start_state):
        self.state = start_state
        self.reward = -1
    
    def take_action(self, state, action):
        if state == 0:
            if all(action == np.array([[0.], [1.]])):
                self.state = 0
            else:
                self.state = 1
        elif state == 1:
            if all(action == np.array([[0.], [1.]])):
                self.state = 2
            else:
                self.state = 1
        elif state == 2:
            if all(action ==np.array([[0.],[1.]])):
                self.state = 1
            else:
                self.state = 3
                
        if state!=3:
            self.reward = -1
        else:
            self.reward = 0  
            
    def is_terminal(self):
        return self.state == 3        

In [263]:
class REINFORCE_mc():
    def __init__(self, Problem, num_episodes=100, y=0.9):
        self.Problem = Problem
        self.num_episodes = num_episodes
        self.y = y
        self.theta = np.zeros((2,1))
        self.x = np.array([[1.,0.],[0.,1.]])
        self.r = []
        self.s = []
        self.a = []
    
    def get_policy_distribution(self):
        return (np.exp(self.theta.T @ self.x)/np.sum(np.exp(self.theta.T @ self.x))).T
    
    def sample_from_distribution(self, P):
        CP = np.cumsum(P)
        rn = np.random.uniform()

        for i in range(len(CP)):
            if CP[i] >= rn:
                return i
    
    def get_action(self):
        action = np.zeros((2,1))
        action[self.sample_from_distribution(self.get_policy_distribution())] = 1 # maybe a separate functon to get the feature vector of action
        return action
    
    def clear(self):
        self.r = []
        self.s = []
        self.a = []
        
    def gen_episode(self,s0=0, max_len=1000):
        self.clear()
        problem = self.Problem(s0)
        
        state = s0
        self.s.append(state)
        
        G = 0
        I = 1
        
        while (not problem.is_terminal()) and max_len != 0:
            action = self.get_action()
            self.a.append(np.argmax(action))
            
            problem.take_action(state, action)
            state = problem.state
            self.s.append(state)
            
            self.r.append(problem.reward)
            G += problem.reward * I
            I *= self.y
            
            max_len -= 1
        
        return G
    
    def mc_without_baseline(self):
        for i in tqdm(range(self.num_episodes)):
            alpha = 0.0001
            G = self.gen_episode(0)
            sub = 0
            I = 1
            
            for t in range(len(self.s)-1):
                G -= sub
                delta_ln = np.copy(self.x[:,self.a[0], np.newaxis])
                pi = self.get_policy_distribution()
                for b in range(2):
                    delta_ln -= np.reshape(pi[b]*self.x[:,b], (2,1))
    
                
                # y^t is covered in G in this implementation. G is not strictly G, rather y^t*G
                self.theta = self.theta + alpha*G*delta_ln 
                
                sub = self.r[t]*I
                I *= self.y
        

In [264]:
a = REINFORCE_mc(shortGridWorld, num_episodes=1000, y=0.9)

In [265]:
a.mc_without_baseline()

In [266]:
a.get_policy_distribution()

array([[0.59061559],
       [0.40938441]])